In [ ]:
import cv2
import math
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.metrics import confusion_matrix

def process_hand_and_detect_fingers(binary_image):
    """
    Detect fingers and thumb in the binary hand image, label them, create lines to fingertips,
    calculate distances, and visualize.
    """
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        print("No contours found!")
        return None, 0, [], False, None

    largest_contour = max(contours, key=cv2.contourArea)
    hull = cv2.convexHull(largest_contour, returnPoints=False)
    defects = cv2.convexityDefects(largest_contour, hull)

    moments = cv2.moments(largest_contour)
    if moments["m00"] != 0:
        cx = int(moments["m10"] / moments["m00"])
        cy = int(moments["m01"] / moments["m00"])
        palm_center = (cx, cy)
    else:
        palm_center = None

    fingertips = []
    if defects is not None:
        for i in range(defects.shape[0]):
            s, e, f, d = defects[i, 0]
            start = tuple(largest_contour[s][0])
            end = tuple(largest_contour[e][0])
            far = tuple(largest_contour[f][0])

            a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
            b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
            c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
            
            cosine_angle = (b ** 2 + c ** 2 - a ** 2) / (2 * b * c)
            cosine_angle = max(-1, min(1, cosine_angle))
            angle = math.acos(cosine_angle)

            if angle < math.radians(90) and d > 10000:
                if start not in fingertips:
                    fingertips.append(start)
                if end not in fingertips:
                    fingertips.append(end)

    if not fingertips and palm_center:
        highest_point = tuple(largest_contour[largest_contour[:, :, 1].argmin()][0])
        fingertips.append(highest_point)

    filtered_fingertips = []
    for fingertip in fingertips:
        if all(math.dist(fingertip, ft) > 20 for ft in filtered_fingertips):
            filtered_fingertips.append(fingertip)

    # Sort fingertips for consistent distance calculation
    fingertips = sorted(filtered_fingertips, key=lambda x: x[0])

    # NEW: Merge fingertips that are too close (distance < 30)
    merged_fingertips = []
    skip_indices = set()
    for i in range(len(fingertips)):
        if i in skip_indices:
            continue
        for j in range(i + 1, len(fingertips)):
            if math.sqrt((fingertips[i][0] - fingertips[j][0]) ** 2 +
                         (fingertips[i][1] - fingertips[j][1]) ** 2) < 50:
                # Merge two fingertips by averaging their coordinates
                merged_fingertip = ((fingertips[i][0] + fingertips[j][0]) // 2,
                                    (fingertips[i][1] + fingertips[j][1]) // 2)
                merged_fingertips.append(merged_fingertip)
                skip_indices.add(j)  # Skip the second fingertip in future iterations
                break
        else:
            merged_fingertips.append(fingertips[i])

    fingertips = merged_fingertips

    distances = []
    thumb_open = False
    thumb_tip = None
    if palm_center:
        for idx, fingertip in enumerate(fingertips, start=1):
            # Calculate distance from palm center to fingertip
            distance = math.sqrt((fingertip[0] - palm_center[0]) ** 2 + (fingertip[1] - palm_center[1]) ** 2)
            distances.append(distance)
    
        # Check thumb detection after distances have been computed
        for idx, (fingertip, distance) in enumerate(zip(fingertips, distances), start=1):
            if idx >= 2:  # Only apply the rule for fingertips classified as 2 or higher
                if fingertip[0] >= 340 or distance < 175:
                    thumb_open = True
                    thumb_tip = fingertip
                    break

    return len(fingertips), thumb_open, thumb_tip, distances, palm_center

def classify_gesture(fingers_detected, thumb_open, distances):
    """
    Classify hand gesture based on the number of fingers detected, thumb status, and distances.
    """
    if fingers_detected == 1:
        if distances[0] < 170:
            return "0"
        elif distances[0] < 200:
            return "6"
        else:
            return "1"
    elif fingers_detected == 2 and not thumb_open:
        return "2"
    elif fingers_detected == 2 and thumb_open:
        return "7"
    elif fingers_detected == 3 and not thumb_open:
        return "3"
    elif fingers_detected == 3 and thumb_open:
        return "8"
    elif fingers_detected == 4 and not thumb_open:
        return "4"
    elif fingers_detected == 4 and thumb_open:
        return "9"
    elif fingers_detected == 5:
        return "5"
    else:
        return "Unknown Gesture"


def process_all_images(directory):
    """
    Process all images in the specified directory and classify gestures.
    """
    results = []
    stats = defaultdict(lambda: {"TP": 0, "FP": 0, "FN": 0})  # To store stats for each class
    total_images = 0
    mismatched_images = 0

    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            filepath = os.path.join(directory, filename)
            image = cv2.imread(filepath)
            
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            lower_blue = np.array([90, 50, 50], dtype=np.uint8)
            upper_blue = np.array([130, 255, 255], dtype=np.uint8)
            background_mask = cv2.inRange(hsv, lower_blue, upper_blue)
            hand_mask = cv2.bitwise_not(background_mask)
            grayscale_hand = cv2.cvtColor(cv2.bitwise_and(image, image, mask=hand_mask), cv2.COLOR_BGR2GRAY)
            _, binary_hand = cv2.threshold(grayscale_hand, 1, 255, cv2.THRESH_BINARY)

            fingers_detected, thumb_open, _, distances, _ = process_hand_and_detect_fingers(binary_hand)
            detected_class = classify_gesture(fingers_detected, thumb_open, distances)

            # Extract the ground truth class from the filename
            ground_truth_class = filename[0]  # Assume the first character in the filename is the class label

            total_images += 1
            # Update stats
            if detected_class == ground_truth_class:
                stats[ground_truth_class]["TP"] += 1  # True Positive
            else:
                mismatched_images += 1
                stats[detected_class]["FP"] += 1  # False Positive
                stats[ground_truth_class]["FN"] += 1  # False Negative

            results.append((filename, detected_class, ground_truth_class))

    # Calculate metrics
    total_correct = sum(stats[c]["TP"] for c in stats)
    total_predictions = sum(stats[c]["TP"] + stats[c]["FP"] for c in stats)
    accuracy = total_correct / total_images if total_images > 0 else 0

    metrics = {"Accuracy": accuracy}
    for c in stats:
        tp = stats[c]["TP"]
        fp = stats[c]["FP"]
        fn = stats[c]["FN"]
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        metrics[c] = {"Precision": precision, "Recall": recall}

    return results, metrics, mismatched_images, total_images

def plot_confusion_matrix_and_metrics(results, total_images, mismatched_images, metrics):
    """
    Generate and display the confusion matrix and metrics in a table format.
    """
    true_classes = [result[2] for result in results]  # Ground truth labels
    predicted_classes = [result[1] for result in results]  # Predicted labels
    class_labels = sorted(set(true_classes + predicted_classes))  # Unique class labels

    # Confusion Matrix
    conf_matrix = confusion_matrix(true_classes, predicted_classes, labels=class_labels)

    # Plot Confusion Matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted Class")
    plt.ylabel("True Class")
    plt.show()

    # Calculate True Positives, False Positives, False Negatives
    metrics_data = []
    for class_label in class_labels:
        tp = sum(1 for true, pred in zip(true_classes, predicted_classes) if true == class_label and pred == class_label)
        fp = sum(1 for true, pred in zip(true_classes, predicted_classes) if true != class_label and pred == class_label)
        fn = sum(1 for true, pred in zip(true_classes, predicted_classes) if true == class_label and pred != class_label)
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        metrics_data.append([class_label, precision, recall, tp, fp, fn])

    # Create metrics DataFrame
    metrics_df = pd.DataFrame(metrics_data, columns=["Class", "Precision", "Recall", "True Positives", "False Positives", "False Negatives"])

    # Add Overall Accuracy row explicitly filling `False Positives` and `False Negatives` columns
    overall_accuracy_row = pd.DataFrame(
        {
            "Class": ["Overall Accuracy"],
            "Precision": [metrics["Accuracy"]],
            "Recall": [None],  # Not applicable for overall accuracy
            "True Positives": [total_images - mismatched_images],
            "False Positives": [sum(row[4] for row in metrics_data)],  # Sum of all False Positives
            "False Negatives": [mismatched_images],  # Total mismatched images
        }
    )
    metrics_df = pd.concat([metrics_df, overall_accuracy_row], ignore_index=True)

    # Display metrics table
    print("\nClassification Metrics Table:")
    display(metrics_df)  # Use display for a cleaner Jupyter output

# Process all images in the "Test" directory
if __name__ == "__main__":
    # Add your original code here: process_hand_and_detect_fingers, classify_gesture, and process_all_images
    test_directory = "./Dataset/Test"  # Specify your test directory
    results, metrics, mismatched_images, total_images = process_all_images(test_directory)
    
    # Print results
    print("\nResults:")
    for filename, detected_class, ground_truth_class in results:
        print(f"{filename}: Detected {detected_class}, Ground Truth {ground_truth_class}")
    
    print("\nMetrics:")
    print(f"Accuracy: {metrics['Accuracy']:.2f}")
    for c in metrics:
        if c != "Accuracy":
            print(f"Class {c}: Precision {metrics[c]['Precision']:.2f}, Recall {metrics[c]['Recall']:.2f}")
    
    # Call the function
    plot_confusion_matrix_and_metrics(results, total_images, mismatched_images, metrics)
    print(f"\nOut of {total_images} images, {mismatched_images} did not match.")


